In [1]:
!pip3 install ensemble_boxes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 14.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 MB 21.9 MB/s eta 0:00:0000:0100:01


In [10]:
import pandas as pd 
from ensemble_boxes import weighted_boxes_fusion as wbf
import numpy as np 
from pycocotools.coco import COCO 
from tqdm import tqdm
from glob import glob

In [11]:
sub_files = sorted(glob('/data/ephemeral/home/submits/*'))
sub_files

['/data/ephemeral/home/submits/output (5).csv',
 '/data/ephemeral/home/submits/output (6).csv',
 '/data/ephemeral/home/submits/output (7).csv',
 '/data/ephemeral/home/submits/output (8).csv',
 '/data/ephemeral/home/submits/output (9).csv']

In [12]:
sub_dfs = [pd.read_csv(sub) for sub in sub_files]

In [13]:
image_ids = sub_dfs[0]['image_id'].tolist()
len(image_ids)

4871

In [14]:
annotation = '/data/ephemeral/home/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [16]:
prediction_strings = []
file_names = []

for i, image_id in tqdm(enumerate(image_ids), total=len(image_ids)):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in sub_dfs:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        # boxes, scores, labels = nms(boxes_list, scores_list, labels_list,iou_thr=iou_thr)
        # boxes, scores, labels = soft_nms(box_list, scores_list, labels_list, iou_thr=iou_thr)
        # boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list,iou_thr=iou_thr)
        # boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list,iou_thr=0.5,conf_type='box_and_model_avg')
        #boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list,iou_thr=0.55)
        boxes, scores, labels = wbf(boxes_list, scores_list, labels_list,iou_thr=0.55)

        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

100%|██████████| 4871/4871 [02:58<00:00, 27.32it/s]


In [9]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble_sample.csv',index=False)

submission.head()

,PredictionString,image_id
0,7 0.8909348845481873 601.72021484375 518.38500...,test/0000.jpg
1,5 0.5472391247749329 131.3141326904297 0.76725...,test/0001.jpg
2,1 0.5726482272148132 422.5028991699219 268.089...,test/0002.jpg
3,9 0.2889176905155182 144.1525115966797 268.030...,test/0003.jpg
4,1 0.33932632207870483 209.483642578125 271.794...,test/0004.jpg
